# Informe nº6.1
    Notas: breve actualización del TFG. Ampliación del método basado en la integral de las luminancias de los píxeles.
    Fecha: 23/07/2020
   
   ### Método de la integral de la luminancia de los píxeles
    Se ha ampliado el proceso por el cual se obtiene la luminancia de los píxeles y el posterior cálculo de la integral.

In [1]:
!pip install opencv-python
!pip install matplotlib
!pip install pvlib
!pip install pyephem
!pip install pandas
!pip install datetime
!pip install sklearn
!pip install seaborn

import cv2
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sn
import math
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pvlib as pv
from datetime import datetime as dt
from glob import glob

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from mascaras import *
from funciones_algoritmo import *
from muestra_imagenes import *

In [2]:
# Lectura de todas las imágenes del repositorio
path_fotos_rad = 'Fotos Cielo/'
images = glob(path_fotos_rad + '*.jpg')

# El rango de imñagenes del que se dispone información sobre la radiación difusa medida por los sesnores es: images[7] a images[38]
path_img = images[30]

# Lectura de la imágen
img_bgr = cv2.imread(path_img, cv2.IMREAD_COLOR)
img_bgr = pre_procesado(img_bgr)

In [3]:
# ELIMINACIÓN DEL DISCO SOLAR

# Se obtiene la hora de la imágen, analizando su nombre
hora_img = hora_imagen(path_img)

# Se obtiene la posición del sol, zenit y azimut, haciendo las correacción correspondiente para transformalos a coordenadas
# de la imágen
zenith_imagen, azimuth_imagen = pos_solar(hora_img, imagen=True)

# Se obtiene el radio teórico que debería tener el disco solar
area_teorica = f_area_zenith()(zenith_imagen / 90)
radio_teorico = np.sqrt(area_teorica / np.pi)

# Se obtiene la posición del sol, centroide, en coordenadas de la imagen

def matriz_distancias(punto):
    # Devuelve una mattriz, del tamaño de la imagen, con la dstancia de cada pixel
    # al punto especificado como parámetro

    pixels_dist = np.array([[pixel_dist([y,x], punto) for y in range(Y)] for x in range(X)])
    
    return pixels_dist

centroide_teorico = centroide_solar(hora_img, imagen=True)
distancias_sol = matriz_distancias(centroide_teorico)

# Se obtiene la máscara que hay que aplicar a la imagen para eliminar las luminancias procedentes del disco solar
mask_sol = cv2.inRange(distancias_sol, 0 , radio_teorico)

In [4]:
# Cálculo de la luminancia de los píxeles
def intensidad_equiponderada(img_bgr):
    # Cálculo de la luminancia del pixeles,
    # como media de los canales B, G y R
    
    B = img_bgr[:,:,0]
    G = img_bgr[:,:,1]
    R = img_bgr[:,:,2]
    
    I = np.array(B/3 + G/3 + R/3, np.uint8)
    return I

    
def intensidad_CCIR(img_bgr):
    # Basado en el convenio CCIR 601 para el cáluclo de 
    # luminancia en imágenes analógicas
    
    B = img_bgr[:,:,0]
    G = img_bgr[:,:,1]
    R = img_bgr[:,:,2]

    I = np.array(B * 0.114 + G * 0.587 + R * 0.299, np.uint8)
    return I 
    
def intensidad_ITUR(img_bgr):
    # Basado en el convenio ITU-R BT 709 para el cáluclo de 
    # luminancia en imágenes digitales
    
    B = img_bgr[:,:,0]
    G = img_bgr[:,:,1]
    R = img_bgr[:,:,2]

    I = np.array(B * 0.0722 + G * 0.7152 + R * 0.2126, np.uint8)
    return I 

In [5]:
# Corrección gamma
def gamma_corr(img_bgr, gamma=2.2, maxVal=255):
    gamma_corr = 1 / gamma
    # Se normaliza la matriz con las imágenes
    img_norm = img_bgr / maxVal
    # Se obtiene la imagen en color corregida y desnormalizada
    img_corr = np.power(img_norm, gamma_corr) * maxVal

    return img_corr

In [6]:
# Obtención de la integral de la luminancia
def integral_intensidad(img_BGR, mask_img=None, metodo='ITUR', coseno=False, gamma=False):
    modos = {'Equiponderado': intensidad_equiponderada,
            'CCIR': intensidad_CCIR,
            'ITUR': intensidad_ITUR}
            
    I = modos[metodo](img_BGR)
    
    # Corrección gamma
    if gamma:
        I = gamma_corr(I, gamma)
    
    # Multiplicación de la matriz de luminancia por la matriz de los ángulos cenitales
    if coseno:
        zenits = matriz_posiciones()[:,:,0]
        I = np.multiply(I, np.cos(np.deg2rad(zenits)))
    
    # Se obtiene el valor acumulado de las intensidades de la imagen
    img_res = cv2.bitwise_and(I,I, mask=mask_img)
    integral = np.sum(img_res)
    
    return integral

In [7]:
mask_no_sol = cv2.bitwise_not(mask_sol, mask=cielo)
integral_res = integral_intensidad(img_bgr, mask_img=mask_no_sol, metodo='ITUR', coseno=False, gamma=2.2)

print('La integral de luminancias obtenida en esta imágen es: {:.2f}'.format(integral_res))

La integral de luminancias obtenida en esta imágen es: 812231689.90


In [8]:
# Se obtiene la radiación difusa medida por los sensores
dataframe_medidas = pd.read_csv('Datos/info_regresion_nublado_integral.csv')

dataframe_medidas.set_index('Unnamed: 0.1', inplace=True)
dataframe_medidas.loc[hora_img.strftime('%Y-%m-%d %H:%M:%S')]

Unnamed: 0                                                  3.365000e+03
Radiacion                                                   3.053211e+02
Radiacion difusa                                            2.710509e+02
Ratio nubes                                                 6.772130e-01
Factor solar                                                0.000000e+00
Área solar                                                  4.066500e+03
Intensidad cielo                                            9.403263e-01
Intensidad cielo ITUR Coseno:True Gamma_corr: True 2.2      4.466193e+08
Intensidad cielo ITUR Coseno:False Gamma_corr: True 2.2     8.176325e+08
Intensidad cielo ITUR Coseno:True Gamma_corr: False 2.2     3.298381e+08
Intensidad cielo ITUR Coseno:False Gamma_corr: False 2.2    5.950647e+08
Intensidad nubes                                            6.141157e-01
ghi                                                         5.599662e+02
Azimuth                                            

In [9]:
# Cálculo del valor de calibración
val_difusa = dataframe_medidas.loc[hora_img.strftime('%Y-%m-%d %H:%M:%S')]['Radiacion difusa']
# val_calib = val_difusa / integral_res
val_calib = integral_res / val_difusa

In [10]:
print('El valor de calibración obtenido en esta imágen es: {:.2f}'.format(val_calib))

El valor de calibración obtenido en esta imágen es: 2996602.04
